In [18]:
import os
import json
import sqlite3
import base64
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [21]:
load_dotenv(override=True)

openai = OpenAI()

MODEL = "gpt-5-nano"

system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [22]:
DB = "prices.db"

ticket_prices = {
    "london": 50000,
    "paris": 25555,
    "tokyo": 20000,
    "sydney": 29999
}

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute(
        "CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)"
    )

    cursor.executemany(
        "INSERT OR REPLACE INTO prices VALUES (?, ?)",
        [(city.lower(), price) for city, price in ticket_prices.items()]
    )

    conn.commit()


In [23]:
def get_ticket_price(destination_city):
    print(f"DATABASE TOOL CALLED: {destination_city}", flush=True)

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            "SELECT price FROM prices WHERE city=?",
            (destination_city.lower(),)
        )
        result = cursor.fetchone()

    if result:
        return f"The return ticket to {destination_city} costs ₹{int(result[0])}."
    
    return "I couldn't find pricing for that city."

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

In [24]:
def artist(city):

    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )

    image_base64 = image_response.data[0].b64_json
    image_bytes = base64.b64decode(image_base64)

    return Image.open(BytesIO(image_bytes))

In [25]:
def talker(message):

    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="onyx",
        input=message
    )

    return response.content

In [26]:
def handle_tool_calls_and_return_cities(message):

    responses = []
    cities = []

    for tool_call in message.tool_calls:

        if tool_call.function.name == "get_ticket_price":

            args = json.loads(tool_call.function.arguments)
            city = args.get("destination_city")

            result = get_ticket_price(city)
            cities.append(city)

            responses.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })

    return responses, cities

In [27]:
def chat(history):

    history = [{"role": h["role"], "content": h["content"]} for h in history]

    messages = [{"role": "system", "content": system_message}] + history

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    cities = []
    image = None

    # tool loop
    while response.choices[0].finish_reason == "tool_calls":

        message = response.choices[0].message

        responses, new_cities = handle_tool_calls_and_return_cities(message)
        cities.extend(new_cities)

        messages.append(message)
        messages.extend(responses)
        
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

    reply = response.choices[0].message.content

    history.append({"role": "assistant", "content": reply})

    voice = talker(reply)

    if cities:
        image = artist(cities[0])

    return history, voice, image

In [28]:
def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]


with gr.Blocks() as ui:

    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)

    with gr.Row():
        audio_output = gr.Audio(autoplay=True)

    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True)

/var/folders/qz/6812z8tj3t59qr8y6wtwzcgw0000gn/T/ipykernel_10507/4223348938.py:8: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=500, type="messages")


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Tokyo
